# PROJETO - Se liga na música

# Conteúdo <a id='back'></a>

* [Introdução](#intro)
* [Etapa 1. Visão geral dos dados](#data_review)
    * [Conclusões](#data_review_conclusions)
* [Etapa 2. Pré-processamento de dados](#data_preprocessing)
    * [2.1 Estilo do cabeçalho](#header_style)
    * [2.2 Valores ausentes](#missing_values)
    * [2.3 Duplicados](#duplicates)
    * [2.4 Conclusões](#data_preprocessing_conclusions)
* [Etapa 3. Teste da hipótese](#hypothesis)
    * [3.1 Hipótese 1: atividade dos usuários nas duas cidades](#activity)
* [Conclusões](#end)

## Introdução <a id='intro'></a>

Neste projeto, vamos comparar as preferências musicais dos habitantes de Springfild e Shelbyville. Iremos estudar os dados de um serviço de streaming de música online para testar a hipótese apresentada abaixo e comparar o comportamento dos usuários dessas duas cidades.

### Objetivo:
Teste a hipótese:
1. A atividade dos usuários é diferente dependendo do dia da semana e da cidade.


### Etapas
Os dados sobre o comportamento dos usuários estão armazenados no arquivo `music_project_en.csv`. Não sabendo como os dados virão do banco de dados, iremos analisá-los para possíveis melhorias.

Primeiro, iremos avaliar a qualidade dos dados e veremos se os problemas são significativos. Depois, durante o pré-processamento dos dados, iremos tratar dos problemas mais críticos.

O projeto consistirá em três etapas:
 1. Visão geral dos dados
 2. Pré-processamento de dados
 3. Teste da hipótese








[Voltar ao Índice](#back)

## Etapa 1. Visão geral dos dados <a id='data_review'></a>



In [2]:
import pandas as pd

**Lendo o arquivo .CSV:**

In [3]:
df = pd.read_csv('music_project_en.csv')

**Imprimindo as primeiras 10 linhas da tabela:**

In [4]:
df.head(10)


,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Shelbyville,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Springfield,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Shelbyville,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Shelbyville,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Springfield,08:34:34,Monday
5,842029A1,Chains,Obladaet,rusrap,Shelbyville,13:09:41,Friday
6,4CB90AA5,True,Roman Messer,dance,Springfield,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Springfield,20:47:49,Wednesday
8,8FA1D3BE,L’estate,Julia Dalia,ruspop,Springfield,09:17:40,Friday
9,E772D5C0,Pessimist,NaN,dance,Shelbyville,21:20:49,Wednesday


**Exibindo informações gerais sobre o DataFrame.**

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63736 non-null  object
 2   artist    57512 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


### Observações: <a id='data_review_conclusions'></a>

Temos o tipo de dado: Objeto.

Os dados são suficientes para testar nossa hipótese, uma vez que o objetivo é: "A atividade dos usuários é diferente dependendo do dia da semana e da cidade." e com a tabela importada através de pandas, eu pude notar que temos todas as linhas e colunas que precisamos.

Alguns problemas que eu notei foram: tabela a notação "NaN" e linha inteira com a mesma notação que podem ser removidas e alguns valores duplicados.

[Voltar ao Índice](#back)

## Etapa 2. Pré-processamento de dados <a id='data_preprocessing'></a>

O objetivo aqui é preparar os dados para a análise.
O primeiro passo é resolver todos os problemas com o cabeçalho. E então podemos passar para os valores ausentes e duplicados. Vamos começar.

Corrija a formatação nos cabeçalhos da tabela.


### Estilo do cabeçalho <a id='header_style'></a>
**Imprimindo os cabeçalhos da tabela (os nomes das colunas):**

In [6]:
print(df.columns)


Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')


Para boas práticas de estilo:
* Todos os caracteres estarão
* Espaços serão excluídos
* Se o nome tiver várias palavras, irei usar "snake_case", que nada mais é que: inicia-se o nome do elemento com letra minúscula e, caso haja mais de uma palavra, elas devem ser separadas por traço baixo/underscore (_)

**1ª maneira de colocar as colunas em letras minúsculas:**

In [7]:
df_col = df.columns.str.lower()
df_col

Index(['  userid', 'track', 'artist', 'genre', '  city  ', 'time', 'day'], dtype='object')

**2ª maneira usando loop para colocar as colunas com letras minúsculas:**

In [8]:
new_col_names = []
for old_name in df.columns:
    name_lowered = old_name.lower()
    new_col_names.append(name_lowered)
df.columns = new_col_names
print(new_col_names)
# Percorrendo os cabeçalhos e convertendo tudo em minúsculos


['  userid', 'track', 'artist', 'genre', '  city  ', 'time', 'day']


**1ª maneira de remover espaços vazios:**

In [9]:
df_strip = df.columns.str.strip()
df_strip

Index(['userid', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')

**2ª maneira usando loop para remover espaços vazios:**

In [10]:
new_col_names = []
for old_name in df.columns:
    name_stripped = old_name.strip()
    new_col_names.append(name_stripped)
df.columns = new_col_names
print(new_col_names)
# Percorrendo os cabeçalhos e removendo os espaços


['userid', 'track', 'artist', 'genre', 'city', 'time', 'day']


**Renomeando a coluna "userdID" segundo as boas práticas de snake_case:**

In [11]:
columns_new ={
    "userid": "user_id"
}
df.rename(columns = columns_new, inplace=True)
df.columns


Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')

[Voltar ao Índice](#back)

### Valores Ausentes <a id='missing_values'></a>
 **Somando as valores ausentes em cada coluna do DataFrame:**

In [12]:
df.isna().sum()


user_id       0
track      1343
artist     7567
genre      1198
city          0
time          0
day           0
dtype: int64

**Substituindo os valores ausentes nas colunas `'track'`, `'artist'` e `'genre'` pela string `'unknown'`**

In [13]:
columns_to_replace = ['track', 'artist', 'genre']
for col in columns_to_replace:
    df[col].fillna('unknown', inplace=True)
# percorrendo os cabeçalhos e substituindo valores ausentes por 'unknown'


**Verificando o resultado para conferir se ainda existem valores ausentes:**

In [14]:
df.isna().sum()


user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64

[Voltar ao Índice](#back)

### Dados Duplicados <a id='duplicates'></a>
**Encontrando os dados duplicados no DataFrame:**

In [15]:
df.duplicated().sum()


3826

**Descartando os dados duplicados explícitos no DataFrame:**

In [16]:
df = df.drop_duplicates().reset_index(drop=True)


**Verificando se todos os dados duplicados explícitos foram excluídos do DataFrame:**

In [17]:
df.duplicated().sum()



0

**Verificando a quantidade de duplicados implícitos na coluna 'genre':**


In [18]:
df['genre'].nunique()

269

**Verificando dados duplicados implícitos na coluna 'genre':**


In [19]:
sorted(df['genre'].unique())

['acid',
 'acoustic',
 'action',
 'adult',
 'africa',
 'afrikaans',
 'alternative',
 'ambient',
 'americana',
 'animated',
 'anime',
 'arabesk',
 'arabic',
 'arena',
 'argentinetango',
 'art',
 'audiobook',
 'avantgarde',
 'axé',
 'baile',
 'balkan',
 'beats',
 'bigroom',
 'black',
 'bluegrass',
 'blues',
 'bollywood',
 'bossa',
 'brazilian',
 'breakbeat',
 'breaks',
 'broadway',
 'cantautori',
 'cantopop',
 'canzone',
 'caribbean',
 'caucasian',
 'celtic',
 'chamber',
 'children',
 'chill',
 'chinese',
 'choral',
 'christian',
 'christmas',
 'classical',
 'classicmetal',
 'club',
 'colombian',
 'comedy',
 'conjazz',
 'contemporary',
 'country',
 'cuban',
 'dance',
 'dancehall',
 'dancepop',
 'dark',
 'death',
 'deep',
 'deutschrock',
 'deutschspr',
 'dirty',
 'disco',
 'dnb',
 'documentary',
 'downbeat',
 'downtempo',
 'drum',
 'dub',
 'dubstep',
 'eastern',
 'easy',
 'electronic',
 'electropop',
 'emo',
 'entehno',
 'epicmetal',
 'estrada',
 'ethnic',
 'eurofolk',
 'european',
 'expe

**Iremos ver os seguintes duplicados implícitos:**
* `hip`
* `hop`
* `hip-hop`

Função sendo executada para corrigir os nomes na coluna 'genre' no DataFrame usando um loop e o método replace para fazer as correções necessárias:

Acima estão os dados duplicados e queremos modificá-los para colocar todos no padrão 'hiphop'.

In [20]:
def replace_wrong_genres(df, column, wrong_genres, correct_genre):
    for wrong_genres in wrong_genres:
        df[column] = df[column].replace(wrong_genres, correct_genre)
    return df

duplicates = ['hip', 'hop', 'hip-hop']
name = 'hiphop'
df = replace_wrong_genres(df, 'genre', duplicates, name)
df['genre'].sample(30)


48366           easy
58070          metal
59822          urban
39251           jazz
22992           rock
41119    alternative
37073         rusrap
43883            pop
40689        melodic
10775          dance
3242           dance
45092         hiphop
15339            pop
9911     psychedelic
41160      classical
58117     soundtrack
43449          dance
58640         rusrap
1812        children
37690            pop
4715           dance
23079          metal
18060          dance
48570          grime
30455            pop
42894     electronic
40284           rock
7064       videogame
19522         glitch
39527            old
Name: genre, dtype: object

**Para certificar que os dados foram corrigidos, iremos usar 'unique' novamente:**

In [21]:
df = sorted(df['genre'].unique())
df

['acid',
 'acoustic',
 'action',
 'adult',
 'africa',
 'afrikaans',
 'alternative',
 'ambient',
 'americana',
 'animated',
 'anime',
 'arabesk',
 'arabic',
 'arena',
 'argentinetango',
 'art',
 'audiobook',
 'avantgarde',
 'axé',
 'baile',
 'balkan',
 'beats',
 'bigroom',
 'black',
 'bluegrass',
 'blues',
 'bollywood',
 'bossa',
 'brazilian',
 'breakbeat',
 'breaks',
 'broadway',
 'cantautori',
 'cantopop',
 'canzone',
 'caribbean',
 'caucasian',
 'celtic',
 'chamber',
 'children',
 'chill',
 'chinese',
 'choral',
 'christian',
 'christmas',
 'classical',
 'classicmetal',
 'club',
 'colombian',
 'comedy',
 'conjazz',
 'contemporary',
 'country',
 'cuban',
 'dance',
 'dancehall',
 'dancepop',
 'dark',
 'death',
 'deep',
 'deutschrock',
 'deutschspr',
 'dirty',
 'disco',
 'dnb',
 'documentary',
 'downbeat',
 'downtempo',
 'drum',
 'dub',
 'dubstep',
 'eastern',
 'easy',
 'electronic',
 'electropop',
 'emo',
 'entehno',
 'epicmetal',
 'estrada',
 'ethnic',
 'eurofolk',
 'european',
 'expe

[Voltar ao Índice](#back)

### Minhas observações <a id='data_preprocessing_conclusions'></a>

Pude notar que os dados vieram com várias questões a serem consertadas e usando python e sua mágica, consegui consertar tudo e iremos para a Etapa 3 de testes de hipótese.

[Voltar ao Índice](#back)

## Etapa 3. Teste da hipótese <a id='hypothesis'></a>

### Hipótese: comparação do comportamento dos usuários em duas cidades <a id='activity'></a>

A hipótese afirma que existem diferenças no consumo de música pelos usuários em Springfield e em Shelbyville nos três dias da semana: segunda-feira (Monday), quarta-feira (Wednesday) e sexta-feira (Friday).


In [ ]:
df.head()

,user_id,track,artist,genre,city,time,day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Shelbyville,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Springfield,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Shelbyville,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Shelbyville,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Springfield,08:34:34,Monday


**Agrupando dados por cidade:**

In [ ]:
df.groupby(by='city')['track'].count()


city
Shelbyville    18512
Springfield    42741
Name: track, dtype: int64

<span style="color:green;">Pode-se notar que os moradores de Springfield ouvem mais que o dobro de músicas que a cidade de Shelbyville durante a semana.</span>

**Agrupando dados por dia da semana:**


In [ ]:
print(df.groupby(by='day')['track'].count())# Calculando as músicas escutadas em cada um desses três dias


day
Friday       21840
Monday       21354
Wednesday    18059
Name: track, dtype: int64


<span style="color:green;">Pode-se notar há um equilíbrio entre os dias da semana em que as músicas são executadas.</span>

**Executando uma função com:
Parâmetros: 'day' e 'city'
e Filtragem consecutiva com indexação lógica.**


In [ ]:
def number_tracks(df, day, city):

    df_filtered = df[(df['day'] == day) & (df['city'] == city)]

    count_tracks = df_filtered['user_id'].count()

    return count_tracks

**Chamando a função `number_tracks()` nos 3 dias da semana aqui especificados e nas duas cidades:**

In [ ]:
number_tracks(df, 'Monday', 'Springfield')

15740

In [ ]:
number_tracks(df, 'Monday', 'Shelbyville')

5614

In [ ]:
number_tracks(df, 'Wednesday', 'Springfield')

11056

In [ ]:
number_tracks(df, 'Wednesday', 'Shelbyville')


7003

In [ ]:
number_tracks(df, 'Friday', 'Springfield')


15945

In [ ]:
number_tracks(df, 'Friday', 'Shelbyville')


5895

**Conclusões**

<span style="color:green;">HIPÓTESE: A atividade dos usuários é diferente dependendo do dia da semana e da cidade.</span>

Se compararmos o compartamento dos ouvintes nas cidades, nota-se que os ouvintes de Springfield ouvem mais músicas durante toda a semana em comparação com a cidade de Shelbyville. Sendo assim, a hipóstese está confirmada.

[Voltar ao Índice](#back)

# Conclusões <a id='end'></a>

Como podemos notar, a hipótese foi totalmente aceita, uma vez que analisando os dados aprensentados, as duas cidades diferem bruscamente em termos de números de ouvintes nos mesmos dias da semana. Nota-se que a cidade de Springfield em todos os dias da semana possui mais ouvintes que a cidade de Shelbyville.

[Voltar ao Índice](#back)